# Determining Best Combination of Preprocessing Techniques on Test Set

## Loading Libraries

In [1]:
%load_ext autoreload
%autoreload 2

# internal imports
import helpers as HL
import glove_module as GV
import neural_nets as NN
import validation_and_prediction as VP
import tokenizing as TO

/Users/HeddaVik/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


## Create the word embeddings using the created gensim-.txt file.

In [2]:
#Pick one, the higher dimension, the better result and longer computational time. 
global_vectors=HL.get_global_vectors(200)

## Creating corpus:
In addition to the acutal corpus, some additional information is needed 

In [3]:
full_corpus, nr_pos_tweets, nr_neg_tweets, total_training_tweets=HL.get_corpus(test=True)

## Picking the neural net
At this stage, we want to use the simple neural net

In [4]:
neural_nets=[NN.basic_model_adam]

## Initializing variables to apply all preprocessing techniques:

In [5]:
#Initializing vectors:
corpuses=[]
corpuses.append(full_corpus)

In [6]:
#Defining names of corpuses: 
names=['original_corpus','SH_corpus','SHM_corpus','H_corpus','HK_corpus','PS_corpus','NS__corpus','OS_corpus','N_corpus','NM_corpus','ST_corpus','SP_corpus','E_corpus','SN_corpus','RS_corpus','EX_corpus','N-2_corpus','N-3_corpus','N-4_corpus']

In [7]:
#Defining inputs to preprocessing function: 
inputs=[{'hashtag': True, 'segmentation_hash': True},
        {'hashtag':True,'segmentation_hash': True,'hashtag_mention':True},
        {'hearts':True},
        {'hugs_and_kisses':True},
        {'pos_smilies':True},
        {'neg_smilies':True},
        {'other_smilies':True},
        {'numbers':True},
        {'numbers':True,'number_mention':True},
        {'stemming':True},
        {'spelling':True},#Warning: When True, it takes app  2.5 h on test set. Recomended to always set to false 
        {'elongation':True},
        {'set_to_not':True},
        {'remove_signs':True},
        {'exclamation':True}]

## Applying all preprocessing techniques to the original corpus: 

In [8]:

for input_ in inputs: 
        corpus=TO.preprocess_corpus(full_corpus, **input_)
        corpuses.append(corpus)
        

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Time in min before starting first for loop: 0.1436559518178304
Time in min after 1  tweets: 0.14389031728108723
Time in min after 25001  tweets: 0.30348013242085775
Time in min after 50001  tweets: 0.43382955392201744
Time in min after 75001  tweets: 0.558660888671875
Time in min after 100001  tweets: 0.688304070631663
Time in min after 125001  tweets: 0.7780535181363424
Time in min after 150001  tweets: 0.8791473507881165
Time in min after 175001  tweets: 0.9658855517705282
Time in min after 200001  tweets: 1.0498501340548196
Time in min total: 1.088587518533071
Reading twitter - 1grams ...
Reading twitter - 2grams ...
Time in min before starting first for loop: 0.14544553756713868
Time in min after 1  tweets: 0.14553653399149577
Time in min after 25001  tweets: 0.36271895170211793
Time in min after 50001  tweets: 0.48187756538391113
Time in min after 75001  tweets: 0.6029905160268147
Time in min after 100001  tweets: 0.7281522

Time in min total: 0.3391306519508362
Time in min before starting first for loop: 1.0331471761067709e-07
Time in min after 1  tweets: 2.296765645345052e-05
Time in min after 25001  tweets: 0.03366755247116089
Time in min after 50001  tweets: 0.06769651571909586
Time in min after 75001  tweets: 0.10239670276641846
Time in min after 100001  tweets: 0.1364606499671936
Time in min after 125001  tweets: 0.1760465661684672
Time in min after 150001  tweets: 0.21609342098236084
Time in min after 175001  tweets: 0.2563955863316854
Time in min after 200001  tweets: 0.2959631681442261
Time in min total: 0.3117439309755961


In [9]:
ns=[2,3,4]
for n in ns: 
    corpus=TO.creating_n_grams_corpus(n,full_corpus)
    corpuses.append(corpus)

## Testing all preprocessing techniques: 

In [ ]:
accuracies=[]
stds=[]

for corpus in corpuses: 
    model_score=VP.classify_with_neural_networks(neural_nets, global_vectors, corpus, total_training_tweets, nr_pos_tweets, epochs=100, n_folds=3)
    accuracies.append(model_score[0][0])
    stds.append(model_score[0][1])

tweets processed: 0  of total number of tweets: 200000
tweets processed: 50000  of total number of tweets: 200000
tweets processed: 100000  of total number of tweets: 200000
tweets processed: 150000  of total number of tweets: 200000
Train on 133332 samples, validate on 66668 samples
Epoch 1/100
133332/133332 [==============================] - 4s 30us/step - loss: 0.4699 - acc: 0.7653 - val_loss: 0.4099 - val_acc: 0.8052
Epoch 2/100
133332/133332 [==============================] - 2s 12us/step - loss: 0.4090 - acc: 0.8072 - val_loss: 0.3968 - val_acc: 0.8118
Epoch 3/100
133332/133332 [==============================] - 2s 12us/step - loss: 0.3957 - acc: 0.8156 - val_loss: 0.3926 - val_acc: 0.8148
Epoch 4/100
133332/133332 [==============================] - 2s 12us/step - loss: 0.3876 - acc: 0.8201 - val_loss: 0.3887 - val_acc: 0.8187
Epoch 5/100
133332/133332 [==============================] - 2s 12us/step - loss: 0.3803 - acc: 0.8241 - val_loss: 0.3862 - val_acc: 0.8182
Epoch 6/100
133

Epoch 4/100
133334/133334 [==============================] - 1s 11us/step - loss: 0.3791 - acc: 0.8237 - val_loss: 0.4056 - val_acc: 0.8102
Epoch 5/100
133334/133334 [==============================] - 1s 11us/step - loss: 0.3728 - acc: 0.8270 - val_loss: 0.4047 - val_acc: 0.8100
Epoch 6/100
133334/133334 [==============================] - 1s 11us/step - loss: 0.3672 - acc: 0.8300 - val_loss: 0.4018 - val_acc: 0.8121
Epoch 7/100
133334/133334 [==============================] - 1s 11us/step - loss: 0.3619 - acc: 0.8335 - val_loss: 0.4029 - val_acc: 0.8131
Epoch 8/100
133334/133334 [==============================] - 2s 11us/step - loss: 0.3575 - acc: 0.8353 - val_loss: 0.4019 - val_acc: 0.8116
Epoch 9/100
133334/133334 [==============================] - 1s 11us/step - loss: 0.3530 - acc: 0.8380 - val_loss: 0.4024 - val_acc: 0.8111
Epoch 00009: early stopping
66666/66666 [==============================] - 2s 30us/step
Model:  basic_model_adam
0.81% (+/- 0.01%)
tweets processed: 0  of total

Epoch 8/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3538 - acc: 0.8381 - val_loss: 0.3958 - val_acc: 0.8107
Epoch 9/100
133334/133334 [==============================] - 2s 14us/step - loss: 0.3497 - acc: 0.8401 - val_loss: 0.3963 - val_acc: 0.8168
Epoch 10/100
133334/133334 [==============================] - 2s 14us/step - loss: 0.3463 - acc: 0.8422 - val_loss: 0.3974 - val_acc: 0.8109
Epoch 00010: early stopping
66666/66666 [==============================] - 2s 35us/step
Train on 133334 samples, validate on 66666 samples
Epoch 1/100
133334/133334 [==============================] - 3s 19us/step - loss: 0.4612 - acc: 0.7694 - val_loss: 0.4231 - val_acc: 0.7981
Epoch 2/100
133334/133334 [==============================] - 2s 12us/step - loss: 0.3960 - acc: 0.8146 - val_loss: 0.4102 - val_acc: 0.8065
Epoch 3/100
133334/133334 [==============================] - 2s 11us/step - loss: 0.3823 - acc: 0.8218 - val_loss: 0.4037 - val_acc: 0.8108
Epoch 4/100
133334/1

133334/133334 [==============================] - 2s 14us/step - loss: 0.3977 - acc: 0.8133 - val_loss: 0.4102 - val_acc: 0.8059
Epoch 3/100
133334/133334 [==============================] - 2s 12us/step - loss: 0.3850 - acc: 0.8197 - val_loss: 0.4052 - val_acc: 0.8077
Epoch 4/100
133334/133334 [==============================] - 2s 12us/step - loss: 0.3765 - acc: 0.8252 - val_loss: 0.4016 - val_acc: 0.8112
Epoch 5/100
133334/133334 [==============================] - 2s 12us/step - loss: 0.3696 - acc: 0.8292 - val_loss: 0.3980 - val_acc: 0.8112
Epoch 6/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3638 - acc: 0.8325 - val_loss: 0.3976 - val_acc: 0.8113
Epoch 7/100
133334/133334 [==============================] - 2s 12us/step - loss: 0.3588 - acc: 0.8355 - val_loss: 0.3957 - val_acc: 0.8104
Epoch 8/100
133334/133334 [==============================] - 2s 12us/step - loss: 0.3536 - acc: 0.8375 - val_loss: 0.3954 - val_acc: 0.8119
Epoch 9/100
133334/133334 [=====

Epoch 7/100
133332/133332 [==============================] - 2s 12us/step - loss: 0.3692 - acc: 0.8297 - val_loss: 0.3839 - val_acc: 0.8186
Epoch 8/100
133332/133332 [==============================] - 2s 13us/step - loss: 0.3647 - acc: 0.8328 - val_loss: 0.3844 - val_acc: 0.8183
Epoch 9/100
133332/133332 [==============================] - 2s 12us/step - loss: 0.3606 - acc: 0.8346 - val_loss: 0.3829 - val_acc: 0.8199
Epoch 10/100
133332/133332 [==============================] - 2s 12us/step - loss: 0.3569 - acc: 0.8368 - val_loss: 0.3880 - val_acc: 0.8181
Epoch 11/100
133332/133332 [==============================] - 2s 12us/step - loss: 0.3535 - acc: 0.8391 - val_loss: 0.3854 - val_acc: 0.8196
Epoch 12/100
133332/133332 [==============================] - 2s 13us/step - loss: 0.3499 - acc: 0.8413 - val_loss: 0.3866 - val_acc: 0.8197
Epoch 00012: early stopping
66668/66668 [==============================] - 3s 39us/step
Train on 133334 samples, validate on 66666 samples
Epoch 1/100
133334

...

133334/133334 [==============================] - 7s 53us/step - loss: 0.4640 - acc: 0.7678 - val_loss: 0.4258 - val_acc: 0.7963
Epoch 2/100
133334/133334 [==============================] - 2s 14us/step - loss: 0.3996 - acc: 0.8124 - val_loss: 0.4124 - val_acc: 0.8048
Epoch 3/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3862 - acc: 0.8195 - val_loss: 0.4063 - val_acc: 0.8093
Epoch 4/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3777 - acc: 0.8242 - val_loss: 0.4034 - val_acc: 0.8118
Epoch 5/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3715 - acc: 0.8282 - val_loss: 0.4028 - val_acc: 0.8113
Epoch 6/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3660 - acc: 0.8309 - val_loss: 0.3994 - val_acc: 0.8141
Epoch 7/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3607 - acc: 0.8341 - val_loss: 0.4012 - val_acc: 0.8141
Epoch 8/100
133334/133334 [=====

Epoch 5/100
133334/133334 [==============================] - 2s 14us/step - loss: 0.3711 - acc: 0.8285 - val_loss: 0.3993 - val_acc: 0.8116
Epoch 6/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3651 - acc: 0.8315 - val_loss: 0.3992 - val_acc: 0.8112
Epoch 7/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3601 - acc: 0.8356 - val_loss: 0.3970 - val_acc: 0.8111
Epoch 8/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3550 - acc: 0.8376 - val_loss: 0.3972 - val_acc: 0.8126
Epoch 9/100
133334/133334 [==============================] - 2s 14us/step - loss: 0.3508 - acc: 0.8401 - val_loss: 0.3967 - val_acc: 0.8164
Epoch 10/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3471 - acc: 0.8423 - val_loss: 0.3965 - val_acc: 0.8130
Epoch 11/100
133334/133334 [==============================] - 2s 14us/step - loss: 0.3434 - acc: 0.8447 - val_loss: 0.3983 - val_acc: 0.8120
Epoch 12/100
13333

Epoch 6/100
133332/133332 [==============================] - 2s 14us/step - loss: 0.3887 - acc: 0.8192 - val_loss: 0.4015 - val_acc: 0.8087
Epoch 7/100
133332/133332 [==============================] - 2s 13us/step - loss: 0.3831 - acc: 0.8220 - val_loss: 0.4002 - val_acc: 0.8104
Epoch 8/100
133332/133332 [==============================] - 2s 13us/step - loss: 0.3790 - acc: 0.8246 - val_loss: 0.4010 - val_acc: 0.8115
Epoch 9/100
133332/133332 [==============================] - 2s 16us/step - loss: 0.3745 - acc: 0.8275 - val_loss: 0.4005 - val_acc: 0.8109
Epoch 10/100
133332/133332 [==============================] - 2s 14us/step - loss: 0.3713 - acc: 0.8293 - val_loss: 0.4085 - val_acc: 0.8060
Epoch 00010: early stopping
66668/66668 [==============================] - 5s 71us/step
Train on 133334 samples, validate on 66666 samples
Epoch 1/100
133334/133334 [==============================] - 7s 50us/step - loss: 0.4747 - acc: 0.7595 - val_loss: 0.4468 - val_acc: 0.7844
Epoch 2/100
133334/1

tweets processed: 50000  of total number of tweets: 200000
tweets processed: 100000  of total number of tweets: 200000
tweets processed: 150000  of total number of tweets: 200000
Train on 133332 samples, validate on 66668 samples
Epoch 1/100
133332/133332 [==============================] - 68s 512us/step - loss: 0.4711 - acc: 0.7646 - val_loss: 0.4107 - val_acc: 0.8044
Epoch 2/100
133332/133332 [==============================] - 2s 13us/step - loss: 0.4094 - acc: 0.8069 - val_loss: 0.3973 - val_acc: 0.8120
Epoch 3/100
133332/133332 [==============================] - 2s 13us/step - loss: 0.3959 - acc: 0.8154 - val_loss: 0.3931 - val_acc: 0.8155
Epoch 4/100
133332/133332 [==============================] - 2s 13us/step - loss: 0.3877 - acc: 0.8203 - val_loss: 0.3893 - val_acc: 0.8178
Epoch 5/100
133332/133332 [==============================] - 2s 13us/step - loss: 0.3802 - acc: 0.8246 - val_loss: 0.3867 - val_acc: 0.8181
Epoch 6/100
133332/133332 [==============================] - 2s 13us

133334/133334 [==============================] - 2s 13us/step - loss: 0.3794 - acc: 0.8237 - val_loss: 0.4057 - val_acc: 0.8099
Epoch 5/100
133334/133334 [==============================] - 2s 14us/step - loss: 0.3730 - acc: 0.8271 - val_loss: 0.4057 - val_acc: 0.8094
Epoch 6/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3676 - acc: 0.8301 - val_loss: 0.4022 - val_acc: 0.8121
Epoch 7/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3622 - acc: 0.8333 - val_loss: 0.4031 - val_acc: 0.8132
Epoch 8/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3578 - acc: 0.8352 - val_loss: 0.4018 - val_acc: 0.8107
Epoch 9/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3533 - acc: 0.8381 - val_loss: 0.4028 - val_acc: 0.8124
Epoch 10/100
133334/133334 [==============================] - 2s 13us/step - loss: 0.3496 - acc: 0.8401 - val_loss: 0.4043 - val_acc: 0.8112
Epoch 11/100
133334/133334 [===

## Want to determine which preprocessing techniques that improved the accuracy, and keep them: 

In [ ]:
corpuses_1=[]
names_1=[]
stds_1=[]
acc_1=[]
print('The original corpus gave accuracy of: ',accuracies[0], 'std:', stds[0],'\n')
for i in range(1,len(accuracies)):
    if accuracies[i]>=accuracies[0]:
        corpuses_1.append(corpuses[i])
        names_1.append(names[i])
        stds_1.append(stds[i])
        acc_1.append(accuracies[i])
        print('IMPROVED:  ',names[i],', score:',accuracies[i],'std:',stds[i])
    else:
        print('Not better:',names[i],', score:',accuracies[i],'std:',stds[i])
        

In [ ]:
# We do not keep SH, as SHM is better. 
# We do not keep N as NM is better 
# We do not keep SP due to time  (158.26 min)
# We do not keep techniques that imporved less than 0.05 percentage points. 

## Apply all techniques that contributed positively

In [ ]:
input_={'hashtag': True, 'segmentation_hash': True, 'hashtag_mention':True,
        'hearts':True,'hugs_and_kisses':True,'elongation':True, 'set_to_not':True}


In [ ]:
#Applying the techniques that we have decided to keep:
best_prepr_corpus=TO.preprocess_corpus(full_corpus, **input_)
best_corpus=HL.creating_n_grams_corpus(2,best_prepr_corpus)

In [ ]:
#Cross validating:
model_score=VP.classify_with_neural_networks(neural_nets, global_vectors, corpus, 
                                             total_training_tweets, nr_pos_tweets, epochs=100, n_folds=3)

In [ ]:
print('Accuracy:',  model_score[0][0], 'std:',model_score[0][1],'\n' )

## Dynamic stopword list

In [ ]:
mindfs= [0,2,5,10,20, 40, 60,100,140,200]
maxdfs=[0.8] 
accuracies_stop=[]
stds_stop=[]
stop_lens=[]
vocabs=[]


for max_ in maxdfs:
    for min_ in mindfs: 
        stopwords, vocab= TO.get_dynamic_stopwords(best_corpus, MinDf=min_, MaxDf=max_)
        stopword_corpus=TO.remove_stopwords(best_corpus, stopwords)
        model_score=VP.classify_with_neural_networks(neural_nets, global_vectors, stopword_corpus, total_training_tweets, nr_pos_tweets, epochs=100, n_folds=3)
        accuracies_stop.append(model_score[0][0])
        stds_stop.append(model_score[0][1])
        stop_lens.append(len(stopwords))
        vocabs.append(vocab)


In [ ]:
for i in range(len(stop_lens)):
    print('min', mindfs[i], 'accuracy', accuracies_stop[i],'std',stds_stop[i],  '\n')
for i in range(len(stop_lens)):
    print('number of stop words', stop_lens[i], 'lenght of vocabolary', len(vocabs[i]),'sum for checking',stop_lens[i]+len(vocabs[i]),  '\n')

## Testing the best combination with the simple and the complex neural net

In [ ]:
stopwords, vocab= TO.get_dynamic_stopwords(best_corpus, MinDf=5, MaxDf=0.8)
final_corpus= TO.remove_stopwords(best_corpus, stopwords)

In [ ]:
model_score=VP.classify_with_neural_networks(NN.basic_model_adam, global_vectors, final_corpus, total_training_tweets, nr_pos_tweets, epochs=100, n_folds=3)

print('Applying the best combination of preprocessing, using pre-trained global vectors with 200 dimensions, an cros-validatino accuracy of', model_score[0][0],'+-',model_score[0][1], 'was achieved using the simple neural net')

In [ ]:
model_score=VP.classify_with_neural_networks(NN.complex_model, global_vectors, final_corpus, total_training_tweets, nr_pos_tweets, epochs=100, n_folds=3)

print('Applying the best combination of preprocessing, using pre-trained global vectors with 200 dimensions, an cros-validatino accuracy of', model_score[0][0],'+-',model_score[0][1], 'was achieved using the compelx neural net')